In [1]:
import redis
import time
from kafka import KafkaProducer

In [2]:
# Redis Config
REDIS_HOST = 'localhost'
REDIS_PORT = 6379
REDIS_KEY = 'traffy_staging_queue' 

# Kafka Config
KAFKA_BROKER = 'localhost:9092'     
KAFKA_TOPIC = 'traffy_data' 
BATCH_SIZE = 5000

In [5]:
try:
    r_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, db=0)
    print(f"✅ Connected to Redis. Queue size: {r_client.llen(REDIS_KEY)}")
except Exception as e:
    print(f"❌ Failed to connect to Redis: {e}")

try:
    producer = KafkaProducer(bootstrap_servers=[KAFKA_BROKER])
    print("✅ Connected to Kafka")
except Exception as e:
    print(f"❌ Failed to connect to Kafka: {e}")

✅ Connected to Redis. Queue size: 0
✅ Connected to Kafka


In [4]:
count = 0
start_time = time.time()

while True:

    raw_data_list = r_client.rpop(REDIS_KEY, count=BATCH_SIZE)

    if not raw_data_list:
        print("🎉 Redis Queue ว่างแล้ว! จบการทำงาน")
        break


    for raw_bytes in raw_data_list:
        producer.send(KAFKA_TOPIC, value=raw_bytes)
    
    count += len(raw_data_list)
    print(f" -> ส่งแล้ว {count} รายการ...", end='\r')

producer.flush()
end_time = time.time()

print(f"\n------------------------------------------------")
print(f"✅ Transfer Completed!")
print(f"📊 Total Records: {count}")
print(f"⏱️ Time Taken: {end_time - start_time:.2f} seconds")
if count > 0:
    print(f"⚡ Speed: {count / (end_time - start_time):.0f} records/sec")

🎉 Redis Queue ว่างแล้ว! จบการทำงาน

------------------------------------------------
✅ Transfer Completed!
📊 Total Records: 787026
⏱️ Time Taken: 344.95 seconds
⚡ Speed: 2282 records/sec
